In [1]:
import time
#from datetime import datetime
from bs4 import BeautifulSoup
import re, os, time
#import json
#from lxml import etree, html
import requests
import pandas as pd
import numpy as np
#import cPickle
#from pymongo import MongoClient
#from lxml import html
import graphlab

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Take in the user input, determine if it works or not.

In [3]:
my_url = "http://steamcommunity.com/id/deckgumby/?xml=0"

### You can still retrieve the IDs of private users, so the next check is when you access the UserInfo API

In [107]:
user_input = raw_input("Paste your Community Prolile URL or 17-digit Steam ID:")
response = requests.get(user_input)
str(re.findall(r"[0-9]{17}", response.text)[0])

Paste your Community Prolile URL or 17-digit Steam ID:http://steamcommunity.com/profiles/76561197982823731/


'76561197982823731'

In [31]:
def get_user_info(uid):
        url = 'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key='\
               +os.environ["ACCESS_STEAM"]+'&steamids='+uid
        
        user = requests.get(url).json()["response"]["players"][0]
        #desired_keys = set(user.keys()) - set(["steamid", "profileurl", "personastateflags", "avatar", "avatarmedium", "steamid"])
        return user

In [65]:
get_user_info("76561198114793191")

{u'avatar': u'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/48/48b5e573f07f7d9ae497915d79ea2a77c07de5f9.jpg',
 u'avatarfull': u'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/48/48b5e573f07f7d9ae497915d79ea2a77c07de5f9_full.jpg',
 u'avatarmedium': u'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/48/48b5e573f07f7d9ae497915d79ea2a77c07de5f9_medium.jpg',
 u'commentpermission': 1,
 u'communityvisibilitystate': 3,
 u'lastlogoff': 1461593048,
 u'loccountrycode': u'GB',
 u'personaname': u'silky',
 u'personastate': 0,
 u'personastateflags': 0,
 u'primaryclanid': u'103582791453877291',
 u'profilestate': 1,
 u'profileurl': u'http://steamcommunity.com/id/topkaka/',
 u'realname': u'\u2665\U0001f495\u2665\u2192',
 u'steamid': u'76561198114793191',
 u'timecreated': 1384542348}

In [95]:
if get_user_info(os.environ["MY_STEAM"])["communityvisibilitystate"] == 1:
    print "Your profile is set to private; we cannot access your gaming preferences!\
                    \nChange your profile settings by going to 'edit > settings > Profile Status and\
                    \nselecting 'Public' if you would like find similar gamers."

Your profile is set to private. We cannot access your gaming preferences!                    
Change your profile settings by going to 'edit > settings > Profile Status and                    
selecting 'Public' if you would like find similar gamers.


In [24]:
class CollectNewUser():
    
    def __init__(self, popular_games):
        self.popular_games = popular_games
        self.uid = self.determine_user_input()
        self.key = os.environ["ACCESS_STEAM"]
        self.bans = None
        self.user = None
        self.friends = None
    
    
    def determine_user_input(self):
        user_input = raw_input("Paste your Steam community profile url here: ")
        if len(user_input) == 17:
            if user_input.isdigit():
                return user_input
            else:
                return "Maybe you tried your 17-digt Steam ID, which was not recognized"

        elif "steamcommunity.com" not in user_input:
            return "Must be a Steam Community URL!"

        else:    
            try:
                response = requests.get(user_input)
                uid = re.findall(r"[0-9]{17}", response.text)[0]
                if len(uid) != 17:
                    return "Url did not work"
                return uid

            except IndexError:
                return "Could not find your profile."

            except requests.ConnectionError:
                return "Could not find your profile."
            
            
    def get_user_info(self):
        """
        For my model, I need the new user's:
          personastate
          location (eventually)
          profile avatar url (eventually)
          other things?
        """
        url = "http://api.steampowered.com/ISteamUser/GetPlayerSummaries"\
              +"/v0002/?key="+self.key+"&steamids="+self.uid
        user = requests.get(url).json()["response"]["players"][0]
        if user["communityvisibilitystate"] == 1:
            return None
        #desired_keys = set(user.keys()) - set(["profileurl", "personastateflags", "avatar", "avatarmedium", "steamid"])
        self.user = {k: user[k] for k in ("personastate", "timecreated", "steamid")}
    
    
    def get_bans(self):
        url = "http://api.steampowered.com/ISteamUser/GetPlayerBans"\
              +"/v1/?key="+self.key+"&steamids="+self.uid
        response = requests.get(url)
        self.s_code = str(response.status_code).startswith("2")
        ban = response.json()["players"][0]
        desired_keys = set(ban.keys()) - set(["SteamId", "DaysSinceLastBan", "EconomyBan",\
                                             "NumberOfGameBans", "NumberOfVACBans"])
        self.bans = {k: ban[k] for k in desired_keys}
    
    
    def get_friends(self):
        url = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key='\
               +self.key+'&steamid='+self.uid+'&relationship=all'
        response = requests.get(url)
        if str(response.status_code).startswith("2"):
            friends = response.json()["friendslist"]["friends"]
            excluding = set(["relationship"])
            #return [{k: friend[k] for k in (set(friend.keys()) - excluding)} for friend in friends]
            self.friends = len(friends)
        else:
            return None
        
        
    def get_game_info(self):
        url =  "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key="\
                +self.key+"&steamid="+self.uid+"&include_appinfo=1\
                &include_played_free_games=1&format=json"
        response = requests.get(url)
        if str(response.status_code).startswith("2"):
            try:
                games = response.json()["response"]["games"]
                for i, game in enumerate(games):
                    desired_keys = set(game.keys()) - set(["has_community_visible_stats",\
                                                           "img_icon_url", "img_logo_url"])
                    if "playtime_2weeks" in desired_keys:
                        game = {k: game[k] for k in desired_keys}
                        games[i] = game
                    else:
                        game = {k: game[k] for k in desired_keys}
                        game[u"playtime_2weeks"] = 0
                        games[i] = game
                return games
            except KeyError:
                return None
        else:
            return None
        
        
    def game_user_frames(self):
        game_df = pd.DataFrame(self.get_game_info())
        owned, played = len(game_df), len(game_df[game_df["playtime_forever"]!=0])
        game_df = game_df[game_df["appid"].isin(self.popular_games)]
        game_df["rating"] = pd.cut(game_df["playtime_forever"],\
                                          bins=[-1, 60, 120, 180, 240, 300, 10e10], \
                                          labels=[0,1,2,3,4,5]).astype(int)
        game_df["user_id"] = ((self.uid+" ") * len(game_df)).split()
        game_df["item_id"] = game_df["appid"].astype(int)
        game_df = game_df[["item_id", "rating", "user_id"]]
        game_df = graphlab.SFrame(game_df)
        user_dict = dict(self.bans.items()
                        + self.user.items()
                        + [("num_friends", self.friends),
                           ("num_played", played),
                           ("num_games", owned)])
        user_df = pd.DataFrame([user_dict])
        user_df["timecreated"] = int(round((time.time() - user_df["timecreated"]) / (3600 * 24 * 365), 2))
        user_df.rename(columns={"steamid":"user_id"}, inplace=True)
        user_df = graphlab.SFrame(user_df)
        return user_df, game_df
    
    

    
        
        
    

In [9]:
game_list = pd.read_csv("/Users/johnnysand/GalvanizeDS/final_project/game_data.csv")

In [10]:
game_list = game_list["appid"].unique()

In [246]:
{"CommunityBanned":new_user.get_bans()["CommunityBanned"],"VACBanned":new_user.get_bans()["CommunityBanned"]}

{'CommunityBanned': False, 'VACBanned': False}

In [25]:
new_user = CollectNewUser(game_list)

Paste your Steam community profile url here: http://steamcommunity.com/id/pocklet/?xml=1


In [26]:
new_user.uid

u'76561198098846168'

In [27]:
new_user.get_bans()
new_user.get_friends()
new_user.get_user_info()

In [28]:
users, games = new_user.game_user_frames()

2016-04-24 09:19:50,399 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1461514788.log


This non-commercial license of GraphLab Create is assigned to johnnysand7@gmail.com and will expire on March 17, 2017. For commercial licensing options, visit https://dato.com/buy/.


In [29]:
games

item_id,rating,user_id
240,5,76561198098846168
4000,5,76561198098846168
400,2,76561198098846168
8190,5,76561198098846168
33910,0,76561198098846168
33930,5,76561198098846168
24780,2,76561198098846168
31280,3,76561198098846168
620,5,76561198098846168
105600,5,76561198098846168


In [30]:
users

CommunityBanned,VACBanned,num_friends,num_games,num_played,personastate,user_id,timecreated
0,0,84,167,138,0,76561198098846168,2


In [415]:
my_games.rename(columns={"steamid":"user_id"}, inplace=True)

In [418]:
graphlab.SFrame(my_games)

CommunityBanned,VACBanned,num_friends,num_games,num_played,personastate,user_id,timecreated
0,0,7,59,34,0,76561197967398882,11


In [ ]:
user_id	CommunityBanned	VACBanned	personastate	timecreated	num_friends	num_games	num_played

In [311]:
new_user.game_frame(my_games, game_list, new_user.uid)

item_id,rating,user_id
10,0,76561197967398882
20,0,76561197967398882
30,0,76561197967398882
50,4,76561197967398882
70,0,76561197967398882
80,0,76561197967398882
220,5,76561197967398882
240,5,76561197967398882
300,0,76561197967398882
320,0,76561197967398882


In [357]:
x = {'a': 1, 'b': 2}
y = {'b': 3, 'c': 4}

In [362]:
dict(x.items() + y.items())

{'a': 1, 'b': 3, 'c': 4}

In [367]:
x.items()

[('a', 1), ('b', 2)]

In [307]:
graphlab.SFrame(my_games)

item_id,rating,user_id
10,0,76561197967398882
20,0,76561197967398882
30,0,76561197967398882
50,4,76561197967398882
70,0,76561197967398882
80,0,76561197967398882
220,5,76561197967398882
240,5,76561197967398882
300,0,76561197967398882
320,0,76561197967398882


In [273]:
user_games["user_id"] = ((new_user.uid+" ") * len(user_games)).split()

In [274]:
user_games

,appid,name,playtime_2weeks,playtime_forever,user_id
0,10,Counter-Strike,0,0,76561197967398882
1,20,Team Fortress Classic,0,0,76561197967398882
2,30,Day of Defeat,0,0,76561197967398882
3,40,Deathmatch Classic,0,0,76561197967398882
4,50,Half-Life: Opposing Force,0,248,76561197967398882
5,60,Ricochet,0,0,76561197967398882
6,70,Half-Life,0,2,76561197967398882
7,130,Half-Life: Blue Shift,0,0,76561197967398882
8,80,Counter-Strike: Condition Zero,0,0,76561197967398882
9,100,Counter-Strike: Condition Zero Deleted Scenes,0,0,76561197967398882


```python
user_id	CommunityBanned	VACBanned	personastate	timecreated	num_friends	num_games	num_played
```

## For the user I need:
profilestate  
banstatus  
numberoffriends  
numberofgames  
numberofplayedgames  

In [145]:
determine_user_input()

Paste your Steam community profile url here: http://steamcommunity.com/storebirds


'Could not find your profile.'

In [ ]:
class NewUser(object):

    def __init__(self, uid):
        """
        Initialize with the Steam API key and Game ID (if needed)
        """
        self.key = os.environ["ACCESS_STEAM"]
        self.uid = None
        self.player_dict = {}
        self.db = None
        self.s_code = None
        self.friend_response = None
        
    
    def user_id(self, uid):
        """
        Resets self.uid to a new 17-digit user ID
        Also checks the GetFriendList API for a status code of 200 (public user)
        or 401 (private user)
        """
        self.uid = uid
        url = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key='\
               +self.key+'&steamid='+self.uid+'&relationship=all'
        #friends = json.loads(urllib2.urlopen(url).read())['friendslist']['friends']
        response = requests.get(url)
        self.s_code = str(response.status_code).startswith("2")
        self.friend_response = response
    
    
    def mongodb_connection(self):
        """
        Connect to MongoDB 'steam'
        """
        client = MongoClient()
        self.db = client.steam
        
        
    def insert_document(self, collection_name="users"):
        """
        Insert a player dictionary document in db_name of MongoDB
        """
        #name = collection_name
        self.db.test.insert_one(self.player_dict)
        
        
    def get_bans(self):
        """
        INPUTS:
          self - .key: API key
                 .uid: 17-digit Steam ID string, up to 100 comma-delimited
        OUTPUTS:
          Dictionary:
            {u'CommunityBanned': False,
             u'DaysSinceLastBan': 0,
             u'EconomyBan': u'none',
             u'NumberOfGameBans': 0, 
             u'NumberOfVACBans': 0,
             u'SteamId': u'76561197967398882', UPDATE - removed
             u'VACBanned': False} VAC is for online gaming
             
        Looks at the 17-digit ID's ban history (up to 100 comma-delimited IDs)
        """
        url = 'http://api.steampowered.com/ISteamUser/GetPlayerBans/v1/?key='+self.key+'&steamids='+self.uid
        #ban = json.loads(urllib2.urlopen(url).read())['players'][0]
        response = requests.get(url)
        self.s_code = str(response.status_code).startswith("2")
        ban = response.json()["players"][0]
        desired_keys = set(ban.keys()) - set(["SteamId"])
        return {k: ban[k] for k in desired_keys}
    
    
    def get_user_info(self):
        """
        INPUTS:
          self - .key: API key
                 .uid: 17-digit Steam ID string, up to 100 comma-delimited 
        OUTPUTS:
          Dictionary:
          {u'avatar': u'https://steamcdn-a.akamaihd.net/.../jpg', # REMOVED
           u'avatarfull': u'https://steamcdn-a.akamaihd.net/.../full.jpg',
           u'avatarmedium': u'https://steamcdn-a.akamaihd.net/.../medium.jpg', # REMOVED
           u'communityvisibilitystate': 3,   # 1 - private profile, 3 - public profile
           u'lastlogoff': 1460189153,  # epoch time
           u'loccountrycode': u'VA',    # May need to look into how to pull that out. Some also have "locstatecode"
           u'personaname': u'Deck Gumby',
           u'personastate': 0,      # 0 - Offline, 1 - Online, 2 - Busy, 3 - Away, 4 - Snooze, 5 - looking to trade, 6 - looking to play.
           u'personastateflags': 0, #
           u'primaryclanid': u'103582791432291036',
           u'profilestate': 1,      # If set, indicates the user has a community profile configured (will be set to '1')
           u'profileurl': u'http://steamcommunity.com/id/deckgumby/',
           u'realname': u'Ken Tucky',
           u'steamid': u'76561197967398882',
           u'timecreated': 1089486992}    # epoch time
        For a user or users (up to 100 comma-delimited),
        return some basic user information
        """
        url = 'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key='\
               +self.key+'&steamids='+self.uid
        #user = json.loads(urllib2.urlopen(url).read())['response']['players'][0]
        user = requests.get(url).json()["response"]["players"][0]
        desired_keys = set(user.keys()) - set(["steamid", "profileurl", "personastateflags", "avatar", "avatarmedium", "steamid"])
        return {k: user[k] for k in desired_keys}
    
    
    def get_friends(self):
        """
        INPUTS:
          self - .key: API key
                 .uid: 17-digit Steam ID
        OUTPUT:
          List of dictionaries, e.g.,
            {u'friend_since': 0,
             u'relationship': u'friend', UPDATE - now removes this field
             u'steamid': u'76561197960559382'}
        Starting with a single user ID, return their friends
        If public, only return users who have public profiles
        """
        #url = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key='\
        #       +self.key+'&steamid='+self.uid+'&relationship=all'
        #friends = json.loads(urllib2.urlopen(url).read())['friendslist']['friends']
        #response = requests.get(url)
        #self.s_code = str(response.status_code).startswith("2")
        friends = self.friend_response.json()["friendslist"]["friends"]
        excluding = set(["relationship"])
        return [{k: friend[k] for k in (set(friend.keys()) - excluding)} for friend in friends]
    
    
    def get_game_info(self):
        """
        INPUTS:
          self - .key: API key
                 .uid: 17-digit Steam ID
        OUTPUT:
          List of dictionaries, e.g.,
            {u'appid': 10,
             u'has_community_visible_stats': True,
             u'img_icon_url': u'6b0312cda02f5f777efa2f3318c307ff9acafbb5', UPDATE - removed
             u'img_logo_url': u'af890f848dd606ac2fd4415de3c3f5e7a66fcb9f', UPDATE - removed
             u'name': u'Counter-Strike',
             u'playtime_forever': 0, in minutes
             u'playtime_2weeks': 0} in minutes UPDATE - added whether played in two weeks or not
             
        Get owned game information for a give user
        Returns a list of dictionaries
        """
        url =  "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key="\
                +self.key+"&steamid="+self.uid+"&include_appinfo=1\
                &include_played_free_games=1&format=json"
        #games = json.loads(urllib2.urlopen(url).read())["response"]["games"]
        games = requests.get(url).json()["response"]
        if "games" in games.keys():
            for i, game in enumerate(games):
                desired_keys = set(game.keys()) - set(["has_community_visible_stats", "img_icon_url", "img_logo_url"])
                if "playtime_2weeks" in desired_keys:
                    game = {k: game[k] for k in desired_keys}
                    games[i] = game
                else:
                    game = {k: game[k] for k in desired_keys}
                    game[u"playtime_2weeks"] = 0
                    games[i] = game
            return games
        else:
            return "none"
    
    
    def get_player_achievments(self, appid):
        """
        Given a player ID and appid, return a list of achievments
        """
        url = "http://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid="+appid+"&key="+self.key+"&steamid="+self.uid
        playerstats = json.loads(urllib2.urlopen(url).read())["playerstats"]
        return {k: playerstats[k] for k in ("achievements", "gameName")}
    
    
    def scrape_profile(self):
        """
        Look at a user's community profile
        Maybe include '?xml=1' at the end of the url?
        """
        url = "http://steamcommunity.com/profiles/"+self.uid
        text = requests.get(url).text
        rm_delims = re.sub(r"[\t\r\n]+", r" ", text)
        rm_breaks = re.sub(r"\<br\>+", r" ", rm_delims).strip()
        soup = BeautifulSoup(rm_breaks, "lxml")
        ### profile summary statement
        profile_summary = " ".join(re.findall(r"[0-9a-zA-Z\.\,\!\?\'\"\$]+", " ".join(\
                                              [list(link.stripped_strings) for link in soup.find_all(\
                                              "div", "profile_header_summary")][0][:-2])))
        
     
        
        
        
        return profile_summary#, profile_items
        
        
        

    

    
    
    def build_dict(self):
        """
        Reference above functions to build a player info dictionary
        Eventually do a self.id kinda thang, naw mean?
        """
        self.player_dict = None
        self.player_dict = {}
        if self.s_code:
            self.player_dict["ban_status"] = self.get_bans()
            self.player_dict["friends"] = self.get_friends()
            self.player_dict["game_info"] = self.get_game_info()
            self.player_dict["user_info"] = self.get_user_info()
            self.player_dict["profile_summary"] = self.scrape_profile()
            self.player_dict["steamid"] = self.uid
        else:
            self.player_dict["ban_status"] = self.get_bans()
            self.player_dict["friends"] = "private"
            self.player_dict["game_info"] = "private"
            self.player_dict["user_info"] = self.get_user_info()
            self.player_dict["profile_summary"] = "private"
            self.player_dict["steamid"] = self.uid
        #self.player_dict["achievments"] = self.get_player_achievments() needs a specific game
    
    
    def check_out(self):
        """
        Returns a built-up dictionary
        """
        return self.player_dict

In [58]:
response = requests.get("http://steamcommunity.com/id/topkaka")
soup = BeautifulSoup(response.content, "lxml")

In [64]:
re.search(r"([0-9]{17})", response.text).group()

u'76561198114793191'

In [63]:
response.content[:10000]

'<!DOCTYPE html>\r\n<html class=" responsive">\r\n<head>\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\r\n\t\t\t<meta name="viewport" content="width=device-width,initial-scale=1">\r\n\t\t<meta name="theme-color" content="#171a21">\r\n\t\t<title>Steam Community :: silky</title>\r\n\t<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon">\r\n\r\n\t<link href="http://steamcommunity-a.akamaihd.net/public/shared/css/motiva_sans.css?v=F3z3QpekjE2f" rel="stylesheet" type="text/css" >\n<link href="http://steamcommunity-a.akamaihd.net/public/shared/css/buttons.css?v=LNGz0aQrH--g" rel="stylesheet" type="text/css" >\n<link href="http://steamcommunity-a.akamaihd.net/public/shared/css/shared_global.css?v=2NieaU9F1tKB" rel="stylesheet" type="text/css" >\n<link href="http://steamcommunity-a.akamaihd.net/public/css/globalv2.css?v=RLlF4NbPEL4H" rel="stylesheet" type="text/css" >\n<link href="http://steamcommunity-a.akamaihd.net/public/css/skin_1/modalContent.css?v=